# Pemisahan data based on hour (Weekdays & Weekend)

In [1]:
import pandas as pd
import googlemaps

import numpy as np

import requests

import openpyxl

import matplotlib.pyplot as plt
import datetime

import seaborn as sns

In [7]:
# Load the Excel file into a pandas DataFrame
df = pd.read_excel('../data/transaction/v1.xlsx')
df.head()

,id,id_telp,email,tanggal,jenis_pesanan,datetime_tugas,lokasi_jemput,lat_jemput,long_jemput,lokasi_tujuan,lat_tujuan,long_tujuan,datetime_selesai,jarak,pendapatan_bersih,poin_tambahan,jenis_pembayaran,jarak_jemput_tujuan,perbedaan_jarak
0,16029,8.132182e+10,purwaramadhika@gmail.com,2023-05-04 00:00:00,Kurir Barang,2023-05-04 05:06:00,"komp gading tutuka 2 blok k8 no 8, Cingcin, Ke...",-7.021726,107.505362,"Jl. Gading Tutuka 2 No.42, Ciluncat, Kec. Cang...",-7.043931,107.540715,2023-05-04 05:31:00,2.50,8800,100.0,Dompet + Tunai,7.490,4.990
1,27243,NaN,deerah60@gmail.com,2023-05-08 00:00:00,Food,2023-05-08 11:23:00,Jl. Dakota 28 Sukaraja Cicendo Bandung,-6.894789,107.569360,"No.154. Jalan Papajaran, Husen Sastranegara",-6.904878,107.579212,2023-05-08 11:51:00,1.89,7200,150.0,Dompet,6.880,4.990
2,14783,8.129765e+10,mrayfan17@gmail.com,2023-05-11 00:00:00,Food,2023-05-11 19:06:00,"Jl. Terusan Jakarta No.6, Babakan Surabaya, Ke...",-6.913613,107.645752,"Jl. Soekarno Hatta, Babakan Penghulu, Bandung ...",-6.937207,107.690535,2023-05-11 19:26:00,4.71,12000,150.0,Tunai,9.698,4.988
3,26411,NaN,daddykurniadi4647@gmail.com,2023-05-02 00:00:00,Food,2023-05-02 19:24:00,GF- Outdoor JL Ciumbuleuit No 147. He...,-6.875969,107.604215,"Jl. Cipaku Indah X No.39, Ledeng, Kec...",-6.859268,107.600874,2023-05-02 19:37:00,2.36,8800,150.0,Dompet,7.347,4.987
4,16842,8.122494e+10,amrytaufikblur@gmail.com,2023-05-14 00:00:00,Ride,2023-05-14 08:00:00,"jl dipatiukur no 107, lebakgede",-6.885659,107.614298,4J4F+4GM jalan gagak dalam,-6.931904,107.627052,2023-05-14 08:09:00,1.99,8000,150.0,Dompet,6.972,4.982


In [18]:
# Separate date and time into separate columns
df['time'] = df['datetime_tugas'].dt.time.apply(lambda x: x.strftime('%H:%M:%S'))  # Convert time to string

In [19]:
# Extract hour from timestamp
df['hour'] = df['datetime_tugas'].dt.hour

In [20]:
df['datetime_tugas'] = pd.to_datetime(df['datetime_tugas'])

# Add a new column for day of the week (0=Monday, 6=Sunday)
df['day_of_week'] = df['datetime_tugas'].dt.dayofweek

# Separate weekends and weekdays
weekdays_df = df[df['day_of_week'] < 5]  # 0-4 represents Monday to Friday
weekends_df = df[df['day_of_week'] >= 5]  # 5-6 represents Saturday and Sunday

Dummy for weekdays

In [21]:
# Add a dummy date (2023-05-01) to the time-only values
weekdays_df['date_dummy'] = pd.to_datetime("2023-05-01 " + weekdays_df['time'])

/tmp/ipykernel_8672/1192847698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekdays_df['date_dummy'] = pd.to_datetime("2023-05-01 " + weekdays_df['time'])


In [23]:
weekdays_df.head()

,id,id_telp,email,tanggal,jenis_pesanan,datetime_tugas,lokasi_jemput,lat_jemput,long_jemput,lokasi_tujuan,...,jarak,pendapatan_bersih,poin_tambahan,jenis_pembayaran,jarak_jemput_tujuan,perbedaan_jarak,time,day_of_week,hour,date_dummy
0,16029,8.132182e+10,purwaramadhika@gmail.com,2023-05-04 00:00:00,Kurir Barang,2023-05-04 05:06:00,"komp gading tutuka 2 blok k8 no 8, Cingcin, Ke...",-7.021726,107.505362,"Jl. Gading Tutuka 2 No.42, Ciluncat, Kec. Cang...",...,2.50,8800,100.0,Dompet + Tunai,7.490,4.990,05:06:00,3,5,2023-05-01 05:06:00
1,27243,NaN,deerah60@gmail.com,2023-05-08 00:00:00,Food,2023-05-08 11:23:00,Jl. Dakota 28 Sukaraja Cicendo Bandung,-6.894789,107.569360,"No.154. Jalan Papajaran, Husen Sastranegara",...,1.89,7200,150.0,Dompet,6.880,4.990,11:23:00,0,11,2023-05-01 11:23:00
2,14783,8.129765e+10,mrayfan17@gmail.com,2023-05-11 00:00:00,Food,2023-05-11 19:06:00,"Jl. Terusan Jakarta No.6, Babakan Surabaya, Ke...",-6.913613,107.645752,"Jl. Soekarno Hatta, Babakan Penghulu, Bandung ...",...,4.71,12000,150.0,Tunai,9.698,4.988,19:06:00,3,19,2023-05-01 19:06:00
3,26411,NaN,daddykurniadi4647@gmail.com,2023-05-02 00:00:00,Food,2023-05-02 19:24:00,GF- Outdoor JL Ciumbuleuit No 147. He...,-6.875969,107.604215,"Jl. Cipaku Indah X No.39, Ledeng, Kec...",...,2.36,8800,150.0,Dompet,7.347,4.987,19:24:00,1,19,2023-05-01 19:24:00
5,7164,8.961537e+10,nugrahahadi110@gmail.com,2023-05-12 00:00:00,Food,2023-05-12 12:32:00,"Jl. Bojonsoang Raya, Bojongsoang, Bandung",-6.983233,107.626535,"Jl. Sapta Taruna Raya No.b-63, Kujangsari, Kec...",...,1.54,7200,150.0,Dompet,6.516,4.976,12:32:00,4,12,2023-05-01 12:32:00


Dummy for weekends

In [22]:
# Add a dummy date (2023-05-01) to the time-only values
weekends_df['date_dummy'] = pd.to_datetime("2023-05-07 " + weekends_df['time'])

/tmp/ipykernel_8672/2048037120.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekends_df['date_dummy'] = pd.to_datetime("2023-05-07 " + weekends_df['time'])


In [32]:
weekends_df.to_csv('../data/hourly/weekends.csv')
weekdays_df.to_csv('../data/hourly/weekdays.csv')

Create file hourly

In [24]:
# Group data by hour for weekdays and weekends
hourly_weekdays = weekdays_df.groupby('hour')
hourly_weekends = weekends_df.groupby('hour')

In [31]:
# Export grouped dataframes to CSV files
for hour, data in hourly_weekdays:
    filename = f'../data/hourly/weekdays/weekdays_hour_{hour}.csv'
    data.to_csv(filename, index=False)
    print(f"Exported {filename}")

for hour, data in hourly_weekends:
    filename = f'../data/hourly/weekends/weekends_hour_{hour}.csv'
    data.to_csv(filename, index=False)
    print(f"Exported {filename}")

Exported ../data/hourly/weekdays/weekdays_hour_0.csv
Exported ../data/hourly/weekdays/weekdays_hour_1.csv
Exported ../data/hourly/weekdays/weekdays_hour_2.csv
Exported ../data/hourly/weekdays/weekdays_hour_3.csv
Exported ../data/hourly/weekdays/weekdays_hour_4.csv
Exported ../data/hourly/weekdays/weekdays_hour_5.csv
Exported ../data/hourly/weekdays/weekdays_hour_6.csv
Exported ../data/hourly/weekdays/weekdays_hour_7.csv
Exported ../data/hourly/weekdays/weekdays_hour_8.csv
Exported ../data/hourly/weekdays/weekdays_hour_9.csv
Exported ../data/hourly/weekdays/weekdays_hour_10.csv
Exported ../data/hourly/weekdays/weekdays_hour_11.csv
Exported ../data/hourly/weekdays/weekdays_hour_12.csv
Exported ../data/hourly/weekdays/weekdays_hour_13.csv
Exported ../data/hourly/weekdays/weekdays_hour_14.csv
Exported ../data/hourly/weekdays/weekdays_hour_15.csv
Exported ../data/hourly/weekdays/weekdays_hour_16.csv
Exported ../data/hourly/weekdays/weekdays_hour_17.csv
Exported ../data/hourly/weekdays/weekd

In [ ]:
df.to_excel('../data/transaction/v2_onlytime.xlsx', index=False)